## 4. Linear ensemble filtering for the Lorenz-96 problem

In this notebook, we apply the stochastic ensemble Kalman filter to the Lorenz-96 problem.

To regularize the inference problem, we use a localization radius `L` to cut-off long-range correlations and improve the conditioning of the covariance matrix. We refers readers to Asch et al. [2] for further details.

[1] Evensen, G., 1994. Sequential data assimilation with a nonlinear quasi‐geostrophic model using Monte Carlo methods to forecast error statistics. Journal of Geophysical Research: Oceans, 99(C5), pp.10143-10162.

[2] Asch, M., Bocquet, M. and Nodet, M., 2016. Data assimilation: methods, algorithms, and applications. Society for Industrial and Applied Mathematics.

[3] Bishop, C.H., Etherton, B.J. and Majumdar, S.J., 2001. Adaptive sampling with the ensemble transform Kalman filter. Part I: Theoretical aspects. Monthly weather review, 129(3), pp.420-436. 

[4] Lorenz, E.N., 1963. Deterministic nonperiodic flow. Journal of atmospheric sciences, 20(2), pp.130-141.

[5] Spantini, A., Baptista, R. and Marzouk, Y., 2019. Coupling techniques for nonlinear ensemble filtering. arXiv preprint arXiv:1907.00389.

In [5]:
using Revise
using LinearAlgebra
using AdaptiveTransportMap
using Statistics
using Distributions

┌ Info: Precompiling AdaptiveTransportMap [bdf749b0-1400-4207-80d3-e689c0e3f03d]
└ @ Base loading.jl:1278
  ** incremental compilation may be fatally broken for this module **

┌ Warning: Type annotations on keyword arguments not currently supported in recipes. Type information has been discarded
└ @ RecipesBase ~/.julia/packages/RecipesBase/92zOw/src/RecipesBase.jl:116
┌ Warning: Type annotations on keyword arguments not currently supported in recipes. Type information has been discarded
└ @ RecipesBase ~/.julia/packages/RecipesBase/92zOw/src/RecipesBase.jl:116


Load some packages to make nice figures

In [6]:
using Plots
default(tickfont = font("CMU Serif", 9), 
        titlefont = font("CMU Serif", 14), 
        guidefont = font("CMU Serif", 12),
        legendfont = font("CMU Serif", 10),
        grid = false)
pyplot()

using LaTeXStrings
PyPlot.rc("text", usetex = "true")
PyPlot.rc("font", family = "CMU Serif")
# gr()
using ColorSchemes

The Lorenz-63  model is a three dimensional system that models the atmospheric convection [4]. This system is a classical benchmark problem in data assimilation. The state $\boldsymbol{x} = (x_1, x_2, x_3)$ is governed by the following set of ordinary differential equations:

\begin{equation}
\begin{aligned}
&\frac{\mathrm{d} x_1}{\mathrm{d} t}=\sigma(x_2-x_1)\\
&\frac{\mathrm{d} x_2}{\mathrm{d} t}=x_1(\rho-x_2)-x_2\\
&\frac{\mathrm{d} x_3}{\mathrm{d} t}=x_1 x_2-\beta x_3,
\end{aligned}
\end{equation}

where $\sigma = 10, \beta = 8/3, \rho = 28$. For these values, the system is chaotic and behaves like a strange attractor.

In this example, we assume that the state is fully observable.

### Simple twin-experiment

Define the dimension of the state and observation vectors

In [7]:
Nx = 40
Ny = 40

40

Define the time steps $\Delta t_{dyn}, \Delta t_{obs}$  of the dynamical and observation models. Observations from the truth are assimilated every $\Delta t_{obs}$.

In [8]:
Δtdyn = 0.05
Δtobs = 0.1

0.1

Define the time span of interest

In [9]:
t0 = 0.0
tf = 100.0
Tf = ceil(Int64, (tf-t0)/Δtobs)

1000

Define the properties of the initial condition

In [10]:
m0 = zeros(Nx)
C0 = Matrix(1.0*I, Nx, Nx);

We construct the state-space representation `F` of the system composed of the deterministic part of the dynamical and observation models. 

The dynamical model is provided by the right hand side of the ODE to solve. For a system of ODEs, we will prefer an in-place syntax `f(du, u, p, t)`, where `p` are parameters of the model.
We rely on `OrdinaryDiffEq` to integrate the dynamical system with the Tsitouras 5/4 Runge-Kutta method adaptive time marching. 

We assume that the state is fully observable, i.e. $h(t,x) = x$.

In [27]:
h(t,x) = x
h(t,x,idx) = x[idx]

F = StateSpace(lorenz96!, h)

StateSpace(AdaptiveTransportMap.lorenz96!, h)

`ϵx` defines the additive process noise applied between the forecast step and the analysis step. The process noise is applied before to sample form the likelihood.

`ϵy` defines the additive observation noise. 

We assume that these noises have Gaussian distribution.

In [12]:
σx = 1e-1
σy = 1.0

ϵx = AdditiveInflation(Nx, zeros(Nx), σx)
ϵy = AdditiveInflation(Ny, zeros(Ny), σy)

AdditiveInflation(40, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … 1.0 0.0; 0.0 0.0 … 0.0 1.0], [1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … 1.0 0.0; 0.0 0.0 … 0.0 1.0])

In [15]:
model = Model(Nx, Ny, Δtdyn, Δtobs, ϵx, ϵy, m0, C0, 0, 0, 0, F);

Set initial condition of the true system

In [16]:
x0 = model.m0 + sqrt(model.C0)*randn(Nx)

40-element Array{Float64,1}:
 -1.0628223227382456
  0.2111821381197456
  0.25174825943904844
  1.641484426732155
  0.6040575596057558
  2.085915794342896
 -2.5356237838972957
 -0.8653459470570214
  1.1650943696880758
 -1.1343597152864833
 -0.06329582977957103
 -0.7439838450290089
 -0.9558554797254603
  ⋮
  0.07929551343453277
  0.6386737832947914
  1.2811819679332144
 -1.625829743299392
 -1.3078878055485963
  0.5502880152693385
 -0.4944653658196742
  0.08281904719985278
 -0.27230083589200377
  0.8882262026863428
 -0.10064579008560609
 -0.6489064413485339

Run dynamics and generate data

In [17]:
data = generate_lorenz96(model, x0, Tf);

LoadError: [91mtype Model has no field f[39m

Define a stochastic ensemble Kalman filter

In [18]:
senkf = StochEnKF(x->x, model.ϵy, model.Δtdyn, model.Δtobs)

Stochastic EnKF  with filtered = false


Define a ensemble transform Kalman filter

In [19]:
etkf = ETKF(x->x, model.ϵy, model.Δtdyn, model.Δtobs, 20*model.Δtobs)

ETKF  with filtered = false


Initialize the ensemble matrix `X` $\in \mathbb{R}^{(N_y + N_x) \times N_e}$.

In [20]:
Ne = 100 #ensemble size
X = zeros(model.Ny + model.Nx, Ne)

# Generate the initial conditions for the state.
X[model.Ny+1:model.Ny+model.Nx,:] .= sqrt(model.C0)*randn(model.Nx, Ne) .+ model.m0

40×100 view(::Array{Float64,2}, 41:80, :) with eltype Float64:
  0.787551    0.582157    0.0517745  …  -1.18444   -0.491106    0.112053
 -3.05701    -0.235147    0.235498      -0.933268  -0.449784   -0.13987
  1.18568    -0.437893   -0.494859       2.03027   -0.287705   -0.690243
  0.288674    0.517632   -0.469376       1.64014    0.616874   -0.134607
 -1.71222     0.587238    1.10808       -1.23874    1.67106    -0.412556
 -0.395118    0.302264   -1.00606    …  -0.1468    -0.261798    0.656139
  0.626456    0.668392    0.480774       1.11467   -1.18127     0.0701848
  0.823939    0.246215   -1.30693        0.939745   0.660961    0.071079
  1.70741    -2.18644     0.0889266     -1.82852   -0.121227    0.279457
  1.07833    -0.454814   -0.0436205     -0.079227   0.757545   -0.207679
 -0.0150352   0.363698   -0.442189   …   1.24065    0.65584    -1.6768
  0.150253    0.524958   -0.215299       1.5689     0.0650527  -0.234715
 -0.252282    0.602504    0.0992384     -1.57302    0.394572   

Apply the sequential filter over the time window

The function `seqassim` provides a friendly API to experiment with the different ensemble filters, the tuning of the different inflation parameters...

In [21]:
Xsenkf = seqassim(F, data, Tf, model.ϵx, senkf, deepcopy(X), model.Ny, model.Nx, t0);

LoadError: [91mUndefVarError: data not defined[39m

In [22]:
Xetkf = seqassim(F, data, Tf, model.ϵx, etkf, deepcopy(X), model.Ny, model.Nx, t0);

LoadError: [91mUndefVarError: data not defined[39m

In [23]:
mean_hist(Xsenkf)

LoadError: [91mUndefVarError: Xsenkf not defined[39m

`mean_hist` stacked together the ensemble mean over the assimilation window.

In [24]:
# Plot the first component of the state over time
nb = 1
ne = size(Xsenkf,1)-1
Δ = 1
plt = plot(xlim = (-Inf, Inf), ylim = (-Inf, Inf), xlabel = L"t", ylabel = L"x_1")
plot!(plt, data.tt[nb:Δ:ne], data.xt[1,nb:Δ:ne], linewidth =  3, color = :teal, label = "True")
plot!(plt, data.tt[nb:Δ:ne], mean_hist(Xsenkf)[1,1+nb:Δ:1+ne], linewidth = 3, grid = false,
     color = :orangered2, linestyle = :dash, label = "sEnKF")
scatter!(plt, data.tt[nb:Δ:ne], data.yt[1,nb:Δ:ne], linewidth = 3, color = :grey, markersize = 5, alpha = 0.5, label = "Observation")
plt

LoadError: [91mUndefVarError: Xsenkf not defined[39m

In [25]:
# Plot the different component of the state over time
nb = 1
ne = size(Xsenkf,1)-1
Δ = 1
plt = plot(layout = grid(3,1), xlim = (-Inf, Inf), ylim = (-Inf, Inf), xlabel = L"t", 
           size = (900, 1000))

for i =1:3
    plot!(plt[i,1], data.tt[nb:Δ:ne], data.xt[i,nb:Δ:ne], linewidth =  2, color = :teal, 
          ylabel = latexstring("x_"*string(i)), legend = (i == 1), label = "True")
    plot!(plt[i,1], data.tt[nb:Δ:ne], mean_hist(Xsenkf)[i,1+nb:Δ:1+ne], linewidth = 2, grid = false,
          color = :orangered2, linestyle = :dash, label = "sEnKF")
    scatter!(plt[i,1], data.tt[nb:Δ:ne], data.yt[i,nb:Δ:ne], linewidth = 3, color = :grey, 
          markersize = 5, alpha = 0.5, label  = "Observation")
end

plt

LoadError: [91mUndefVarError: Xsenkf not defined[39m